# 02 Make Network Version 01
Step 2 applies project cards, makes Python changes, and writes out a pickle of the version 01 network

In [1]:
import os
import sys
import yaml
from subprocess import Popen
import pickle

import pandas as pd
import geopandas as gpd
import numpy as np


from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import WranglerLogger
from network_wrangler import ProjectCard

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO)

### Remote I/O and Parameters

In [4]:
input_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'client_met_council_network')
pickle_dir = os.path.join(input_dir, 'network_standard', 'pickles')
output_dir = os.path.join(input_dir, 'network_standard')
card_dir = os.path.join(input_dir, 'project_cards')
lasso_dir = os.path.join('/Users', 'wsp', 'Documents', 'GitHub', 'lasso')

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2020-08-22 20:50:26, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso
2020-08-22 20:50:26, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/lasso


### Read Version 00 Scenario

In [7]:
version_01_pickle_file_name = os.path.join(pickle_dir, 'working_scenario_01.pickle')
v_02_scenario = pickle.load(open(version_01_pickle_file_name, 'rb'))

### Apply Project Cards

In [ ]:
project_card_list = []
#card_01_file_name = os.path.join(card_dir, 'year_2018_corrections_assign_group_roadway_class.yml')
#card_02_file_name = os.path.join(card_dir, 'year_2018_corrections_assign_group.yml')

#card = ProjectCard.read(card_01_file_name, validate = False)
#project_card_list.append(card)

#card = ProjectCard.read(card_02_file_name, validate = False)
#project_card_list.append(card)

In [ ]:
for project_card in project_card_list:
    v_02_scenario.apply_project(project_card)

In [ ]:
v_02_scenario.applied_projects

## Make Travel Model Network

In [ ]:
m_net = ModelRoadwayNetwork.from_RoadwayNetwork(
    roadway_network_object = v_02_scenario.road_net, 
    parameters = parameters)

In [ ]:
m_net.roadway_standard_to_met_council_network()

## Write to Disk

### As Shapefile

In [ ]:
m_net.write_roadway_as_shp(
    output_link_shp = os.path.join(output_dir, 'v02_links.shp'),
    output_node_shp = os.path.join(output_dir, 'v02_nodes.shp'),
)

### As Fixed width (for Cube)

In [ ]:
m_net.write_roadway_as_fixedwidth(
    output_link_txt = os.path.join(output_dir, 'cube', 'v02_links.txt'),
    output_node_txt = os.path.join(output_dir, 'cube', 'v02_nodes.txt'),
    output_link_header_width_txt = os.path.join(output_dir, 'links_header_width.txt'),
    output_node_header_width_txt = os.path.join(output_dir, 'nodes_header_width.txt'),
    output_cube_network_script = os.path.join(output_dir, 'make_complete_network_from_fixed_width_file.s'),
)

In [ ]:
standard_transit_net = StandardTransit.fromTransitNetwork(v_01_scenario.transit_net)
standard_transit_net.write_as_cube_lin(outpath = os.path.join(output_dir, "v02_transit.lin"))

### As Network Standard

In [ ]:
v_01_scenario.road_net.write(path = output_dir, filename = 'v02_')

In [ ]:
## TODO: update links.df and in v_01_scenario and write out as pickle
## create 03, which reads in v_01_scenario and makes v_02

### Scenario as Pickle

In [ ]:
scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(v_01_scenario, open(scenario_filename, 'wb'))